# Helper functions:

In [1]:
import boto3
import json
from botocore.exceptions import ClientError

regions = [
    'us-east-1',  # US East (N. Virginia)
    'us-east-2',  # US East (Ohio)
    'us-west-1',  # US West (N. California)
    'us-west-2',  # US West (Oregon)
    'ap-south-1',  # Asia Pacific (Mumbai)
    'ap-northeast-3',  # Asia Pacific (Osaka)
    'ap-northeast-2',  # Asia Pacific (Seoul)
    'ap-southeast-1',  # Asia Pacific (Singapore)
    'ap-southeast-2',  # Asia Pacific (Sydney)
    'ap-northeast-1',  # Asia Pacific (Tokyo)
    'ca-central-1',  # Canada (Central)
    'eu-central-1',  # Europe (Frankfurt)
    'eu-west-1',  # Europe (Ireland)
    'eu-west-2',  # Europe (London)
    'eu-west-3',  # Europe (Paris)
    'eu-north-1',  # Europe (Stockholm)
    'sa-east-1'   # South America (São Paulo)
]

#helper functions
def get_amazon_linux_2023_ami(region):
    try:
        ec2_client = boto3.client('ec2', region_name=region)
        response = ec2_client.describe_images(
            Filters=[
                {
                    'Name': 'description',
                    'Values': ['Amazon Linux 2023 AMI*']
                },
                {
                    'Name': 'architecture',
                    'Values': ['x86_64']
                }
            ], 
            Owners=['amazon']
            # Sorting by creation date to get the latest AMI
        )
        if 'Images' in response and len(response['Images']) > 0:
            # Assuming the latest image is the first in the list after sorting by creation date
            latest_image = sorted(response['Images'], key=lambda x: x['CreationDate'], reverse=True)[0]
            return latest_image['ImageId']
        else:
            return None
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None


def create_security_group(vpc, ec2):
    try:
        # Check for existing security groups with the same name in the VPC
        existing_sgs = ec2.security_groups.filter(
            Filters=[
                {'Name': 'vpc-id', 'Values': [vpc.id]},
                {'Name': 'group-name', 'Values': ['allow-all-traffic']}
            ]
        )
        duplicated_sgs = [sg for sg in existing_sgs]
        
        if duplicated_sgs:
            print("Duplicated security groups found:")
            for sg in duplicated_sgs:
                print(f" - {sg.group_id} ({sg.group_name})")
            return duplicated_sgs
        
        # Create a new security group if no duplicates found
        sg = ec2.create_security_group(
            Description='Allow all inbound and outbound traffic',
            GroupName='allow-all-traffic',
            VpcId=vpc.id
        )
        try:
            sg.authorize_ingress(
                IpPermissions=[{
                    'IpProtocol': '-1',
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }]
            )
        except ClientError as e:
            print(f"An error occurred when setting ingress: {e}")
        try:
            sg.authorize_egress(
                IpPermissions=[{
                    'IpProtocol': '-1',
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }]
            )
        except ClientError as e:
            print(f"An error occurred when setting egress: {e}")
        return sg
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None
    


def create_vpc(ec2, CidrBlock, region_suffix):
    try:
        vpc = ec2.create_vpc(CidrBlock=CidrBlock)
        vpc.wait_until_available()
        vpc.create_tags(Tags=[{"Key": "Name", "Value": f"VPC_waveSpreader_experiment_{region_suffix}"}])
        return vpc
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def create_subnet(vpc, ec2, cidr_block):
    try:
        subnet = ec2.create_subnet(CidrBlock=cidr_block, VpcId=vpc.id)
        subnet.create_tags(Tags=[{"Key": "Name", "Value": f"Subnet_waveSpreader_experiment_{vpc.id}"}])
        return subnet
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def create_internet_gateway(vpc, ec2):
    try:
        igw = ec2.create_internet_gateway()
        vpc.attach_internet_gateway(InternetGatewayId=igw.id)
        return igw
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def create_route_table(vpc, igw):
    try:
        route_table = vpc.create_route_table()
        route_table.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId=igw.id)
        return route_table
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

def associate_route_table(route_table, subnet):
    try:
        route_table.associate_with_subnet(SubnetId=subnet.id)
    except ClientError as e:
        print(f"An error occurred: {e}")

# Function to create a VPC peering connection
def create_vpc_peering_connection(ec2, vpc_id, peer_vpc_id, peer_region):
    try:
        peering_connection = ec2.create_vpc_peering_connection(
            VpcId=vpc_id,
            PeerVpcId=peer_vpc_id,
            PeerRegion=peer_region
        )
        if 'VpcPeeringConnection' in peering_connection:
            return peering_connection['VpcPeeringConnection']
        else:
            print("Peering connection creation failed.")
            return None
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

# Function to accept a VPC peering connection
def accept_vpc_peering_connection(ec2_resource, peering_connection_id):
    try:
        peering_connection = ec2_resource.VpcPeeringConnection(peering_connection_id)
    except ClientError as e:
        print(f"An error occurred: {e}")
    try:
        peering_connection.accept()
    except ClientError as e:
        print(f"An error occurred: {e}")


def update_route_tables_for_peering(route_table, peering_connection_id, peer_cidr):
    try:
        route_table.create_route(
            DestinationCidrBlock=peer_cidr,
            VpcPeeringConnectionId=peering_connection_id
        )
    except ClientError as e:
        print(f"An error occurred: {e}")
        
def delete_internet_gateways(ec2):
    try:
        for igw in ec2.internet_gateways.all():
            for attachment in igw.attachments:
                igw.detach_from_vpc(VpcId=attachment['VpcId'])
            igw.delete()
            print(f"Deleted Internet Gateway: {igw.id}")
    except ClientError as e:
        print(f"An error occurred while deleting Internet Gateway: {e}")

def delete_route_tables(ec2):
    try:
        for rt in ec2.route_tables.all():
            if not rt.associations_attribute:
                rt.delete()
                print(f"Deleted Route Table: {rt.id}")
    except ClientError as e:
        print(f"An error occurred while deleting Route Table: {e}")

def delete_vpc_peering_connections(ec2):
    try:
        for pcx in ec2.vpc_peering_connections.all():
            if pcx.status['Code'] != 'deleting':
                pcx.delete()
                print(f"Deleted VPC Peering Connection: {pcx.id}")
    except ClientError as e:
        print(f"An error occurred while deleting VPC Peering Connection: {e}")

def delete_user_managed_prefix_lists(ec2_client):
    try:
        prefix_lists = ec2_client.describe_managed_prefix_lists()['PrefixLists']
        for pl in prefix_lists:
            if not pl['PrefixListName'].startswith('com.amazonaws'):
                ec2_client.delete_managed_prefix_list(PrefixListId=pl['PrefixListId'])
                print(f"Deleted Managed Prefix List: {pl['PrefixListId']}")
    except ClientError as e:
        print(f"An error occurred while deleting Managed Prefix List: {e}")
        
def delete_instances(instance_ids,default_region='eu-north-1'):
  # Specify a default region to create the initial client
    try:
        # Create a default EC2 client to get the instance details
        ec2_client = boto3.client('ec2', region_name=default_region)
        # Describe the instances to get their details
        response = ec2_client.describe_instances(InstanceIds=instance_ids)
        logger.info(f"Describe instances response: {response}")
        
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                instance_id = instance['InstanceId']
                availability_zone = instance['Placement']['AvailabilityZone']
                region = get_region_from_az(availability_zone)
                #logger.info(f"Instance {instance_id} is in availability zone {availability_zone}, region {region}")
                
                if region:
                    try:
                        # Create an EC2 client for the specific region
                        ec2 = boto3.client('ec2', region_name=region)
                        # Terminate the instance
                        ec2.terminate_instances(InstanceIds=[instance_id])
                        logger.info(f"Terminated instance {instance_id} in region {region}")
                    except Exception as e:
                        logger.error(f"Failed to terminate instance {instance_id}: {str(e)}")
                else:
                    logger.error(f"Could not determine a valid region from availability zone {availability_zone} for instance {instance_id}")
    except Exception as e:
        logger.error(f"Failed to describe instances: {str(e)}")

def delete_vpc(ec2, vpc_id):
    try:
        vpc = ec2.Vpc(vpc_id)
        
        # Detach and delete internet gateways
        for igw in vpc.internet_gateways.all():
            vpc.detach_internet_gateway(InternetGatewayId=igw.id)
            igw.delete()
        
        # Delete subnets
        for subnet in vpc.subnets.all():
            subnet.delete()
        
        # Delete route tables
        for rt in vpc.route_tables.all():
            if not rt.associations_attribute:
                rt.delete()
        
        # Delete security groups
        for sg in vpc.security_groups.all():
            if sg.group_name != 'default':
                sg.delete()
        
        # Delete VPC peering connections
        for pcx in ec2.vpc_peering_connections.all():
            if pcx.requester_vpc.id == vpc_id or pcx.accepter_vpc.id == vpc_id:
                pcx.delete()
        
        # Finally, delete the VPC
        vpc.delete()
        print(f"Deleted VPC: {vpc_id}")
    except ClientError as e:
        print(f"An error occurred: {e}")
        
def create_instance(ec2, ami_id, subnet_id, private_ip, security_group_id, user_data):
    try:
        instance = ec2.create_instances(
            ImageId=ami_id,
            InstanceType='t3.micro',
            MinCount=1,
            MaxCount=1,
            NetworkInterfaces=[{
                'DeviceIndex': 0,
                'AssociatePublicIpAddress': True,
                'SubnetId': subnet_id,
                'PrivateIpAddress': private_ip,
                'Groups': [security_group_id],
            }],
            IamInstanceProfile={
                'Name': 'Experiment_'  # Replace with the name of your IAM role
            },
            UserData=user_data
        )[0]
        return instance
    except ClientError as e:
        logger.error(f"Error creating instance: {str(e)}")
        return None
def delete_vpc_dependencies(vpc_id,region):
    ec2 = boto3.client('ec2',region_name=region)
    
    # Delete Subnets
    subnets = ec2.describe_subnets(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['Subnets']
    for subnet in subnets:
        ec2.delete_subnet(SubnetId=subnet['SubnetId'])
        print(f"Deleted Subnet: {subnet['SubnetId']}")

    # Delete Route Tables
    route_tables = ec2.describe_route_tables(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['RouteTables']
    for route_table in route_tables:
        if not route_table['Associations'][0]['Main']:  # Skip the main route table
            ec2.delete_route_table(RouteTableId=route_table['RouteTableId'])
            print(f"Deleted Route Table: {route_table['RouteTableId']}")

    # Delete Network Interfaces
    network_interfaces = ec2.describe_network_interfaces(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['NetworkInterfaces']
    for interface in network_interfaces:
        ec2.delete_network_interface(NetworkInterfaceId=interface['NetworkInterfaceId'])
        print(f"Deleted Network Interface: {interface['NetworkInterfaceId']}")

    # Delete Internet Gateways
    internet_gateways = ec2.describe_internet_gateways(Filters=[{'Name': 'attachment.vpc-id', 'Values': [vpc_id]}])['InternetGateways']
    for gateway in internet_gateways:
        ec2.detach_internet_gateway(InternetGatewayId=gateway['InternetGatewayId'], VpcId=vpc_id)
        ec2.delete_internet_gateway(InternetGatewayId=gateway['InternetGatewayId'])
        print(f"Deleted Internet Gateway: {gateway['InternetGatewayId']}")

    # Delete NAT Gateways
    nat_gateways = ec2.describe_nat_gateways(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['NatGateways']
    for nat_gateway in nat_gateways:
        ec2.delete_nat_gateway(NatGatewayId=nat_gateway['NatGatewayId'])
        print(f"Deleted NAT Gateway: {nat_gateway['NatGatewayId']}")

    # Delete Security Groups
    security_groups = ec2.describe_security_groups(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['SecurityGroups']
    for security_group in security_groups:
        if security_group['GroupName'] != 'default':  # Skip the default security group
            ec2.delete_security_group(GroupId=security_group['GroupId'])
            print(f"Deleted Security Group: {security_group['GroupId']}")

    # Delete VPN Connections and Gateways
    vpn_connections = ec2.describe_vpn_connections(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['VpnConnections']
    for vpn_connection in vpn_connections:
        ec2.delete_vpn_connection(VpnConnectionId=vpn_connection['VpnConnectionId'])
        print(f"Deleted VPN Connection: {vpn_connection['VpnConnectionId']}")

    vpn_gateways = ec2.describe_vpn_gateways(Filters=[{'Name': 'attachment.vpc-id', 'Values': [vpc_id]}])['VpnGateways']
    for vpn_gateway in vpn_gateways:
        ec2.delete_vpn_gateway(VpnGatewayId=vpn_gateway['VpnGatewayId'])
        print(f"Deleted VPN Gateway: {vpn_gateway['VpnGatewayId']}")

    # Delete Endpoints
    endpoints = ec2.describe_vpc_endpoints(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])['VpcEndpoints']
    for endpoint in endpoints:
        ec2.delete_vpc_endpoints(VpcEndpointIds=[endpoint['VpcEndpointId']])
        print(f"Deleted VPC Endpoint: {endpoint['VpcEndpointId']}")

    # Delete Peering Connections
    peering_connections = ec2.describe_vpc_peering_connections(Filters=[{'Name': 'requester-vpc-info.vpc-id', 'Values': [vpc_id]}])['VpcPeeringConnections']
    for peering_connection in peering_connections:
        ec2.delete_vpc_peering_connection(VpcPeeringConnectionId=peering_connection['VpcPeeringConnectionId'])
        print(f"Deleted VPC Peering Connection: {peering_connection['VpcPeeringConnectionId']}")

    # Finally, delete the VPC
    ec2.delete_vpc(VpcId=vpc_id)
    print(f"Deleted VPC: {vpc_id}")
def delete_vpcs():
    for region in regions:
        print (region)
        ec2 = boto3.resource('ec2', region_name=region)
        ec2client = boto3.client('ec2', region_name=region)
        # Retrieve all VPCs
        vpcs = ec2.vpcs.all()

        for vpc in vpcs:
            # Delete VPC peering connections
            peering_connections = ec2client.describe_vpc_peering_connections()['VpcPeeringConnections']
            for peering in peering_connections:
                peering_id = peering['VpcPeeringConnectionId']
                peering_state = peering['Status']['Code']
                try:
                    ec2client.delete_vpc_peering_connection(VpcPeeringConnectionId=peering_id)
                except ClientError as e:
                        print(f'Failed to delete vpc_peering_connection  {peering_id}: {e}')
            # Detach and delete internet gateways
            for igw in vpc.internet_gateways.all():
                print(f'Detaching and deleting internet gateway {igw.id}')
                igw.detach_from_vpc(VpcId=vpc.id)
                igw.delete()

            # Delete route table associations
            for rt in vpc.route_tables.all():
                for association in rt.associations:
                    if not association.main:
                        print(f'Deleting route table association {association.id}')
                        association.delete()
                if not rt.associations:
                    print(f'Deleting route table {rt.id}')
                    rt.delete()

            # Delete security groups (skip the default one)
            # Delete security groups (skip the default one)
            for sg in vpc.security_groups.all():
                if sg.group_name != 'default':
                    print(f'Deleting security group {sg.id}')
                    try:
                        sg.delete()
                    except ClientError as e:
                        print(f'Failed to delete security group {sg.id}: {e}')

            # Delete network ACLs (skip the default one)
            for nacl in vpc.network_acls.all():
                if not nacl.is_default:
                    print(f'Deleting network ACL {nacl.id}')
                    try:
                        nacl.delete()
                    except ClientError as e:
                        print(f'Failed to delete network ACL {nacl.id}: {e}')
                    

            # Delete subnets
            for subnet in vpc.subnets.all():
                print(f'Deleting subnet {subnet.id}')
                try:
                    subnet.delete()
                except ClientError as e:
                    print(f'Failed to delete network ACL {subnet.id}: {e}')
                

            # Finally, delete the VPC
    for region in regions:
            print (region)
            ec2 = boto3.resource('ec2', region_name=region)
            ec2client = boto3.client('ec2', region_name=region)
            # Retrieve all VPCs
            vpcs = ec2.vpcs.all()
            try:
                vpc.delete()
            except ClientError as e:
                    print(f'Failed to delete VPC {vpc.id}: {e}')


def get_security_group_id(sg_group_dict):
    sg_group = sg_group_dict.get('sg_group')
    if sg_group:
        sg_id = sg_group.id
        return sg_id
    else:
        raise ValueError("No 'sg_group' key found in the dictionary")
def increment_ip(ip_address,i):
    # Split the IP address into its components
    parts = ip_address.split('.')
    # Convert the components to integers
    parts = [int(part) for part in parts]
    
    # Increment the last part of the IP address
    parts[3] += i
    
    # Handle overflow from the last part to the earlier parts
    for i in range(3, -1, -1):
        if parts[i] > 255:
            parts[i] = 0
            if i != 0:
                parts[i-1] += 1
    
    # Convert the parts back to strings
    new_ip_address = '.'.join(str(part) for part in parts)
    return new_ip_address

Please copy the vpc_info from "Testbed_setup.ipynb" after generating it.

In [2]:
vpc_info=json.loads('''{"us-east-1": {"vpc_id": "vpc-00c10b12ae9be8283", "route_table_id": "rtb-0e4d05861e562769b", "cidr_block": "10.0.1.0/24", "subnet_id": "subnet-0235a509faf4c3724", "sg_group": "sg-0b45c49f0d3066694"}, "us-east-2": {"vpc_id": "vpc-05746bd59016fa9bd", "route_table_id": "rtb-0bea3b4f3996adaf2", "cidr_block": "10.0.2.0/24", "subnet_id": "subnet-0005a4b7682ed9823", "sg_group": "sg-0c252ad483afa0e78"}, "us-west-1": {"vpc_id": "vpc-0bf1c4bde20e87f6c", "route_table_id": "rtb-04c9ca602d286b0c4", "cidr_block": "10.0.3.0/24", "subnet_id": "subnet-069cbf605e573d8b2", "sg_group": "sg-0fe2d839448179c63"}, "us-west-2": {"vpc_id": "vpc-0aa5cdaa04869c3eb", "route_table_id": "rtb-062cd26b20f42809e", "cidr_block": "10.0.4.0/24", "subnet_id": "subnet-07f323295a6580a9d", "sg_group": "sg-0d172a13ca4408f01"}, "ap-south-1": {"vpc_id": "vpc-0d4438731a5a62503", "route_table_id": "rtb-042e064e7ef23e0ec", "cidr_block": "10.0.5.0/24", "subnet_id": "subnet-0cabd266661f459ff", "sg_group": "sg-0193a6f1e8a925934"}, "ap-northeast-3": {"vpc_id": "vpc-0b0d11e5728ff22ea", "route_table_id": "rtb-06bb711f43942ba2e", "cidr_block": "10.0.6.0/24", "subnet_id": "subnet-0c6c00975c7d99ba3", "sg_group": "sg-02287368446c2c623"}, "ap-northeast-2": {"vpc_id": "vpc-07cb9f3a6631ab8ea", "route_table_id": "rtb-0a7f62d4449e74968", "cidr_block": "10.0.7.0/24", "subnet_id": "subnet-0f1e2dec3ca914618", "sg_group": "sg-06a3b8163080b9ab9"}, "ap-southeast-1": {"vpc_id": "vpc-09fee8a03082bc0d5", "route_table_id": "rtb-0764bb8780452991e", "cidr_block": "10.0.8.0/24", "subnet_id": "subnet-01db403c810bdbf28", "sg_group": "sg-0430565d8a16d2ece"}, "ap-southeast-2": {"vpc_id": "vpc-06c8782d2d7b4b307", "route_table_id": "rtb-0826e6cc95a3a4548", "cidr_block": "10.0.9.0/24", "subnet_id": "subnet-05c481e1a0ad58a30", "sg_group": "sg-07666f944900f2956"}, "ap-northeast-1": {"vpc_id": "vpc-0e6145013b3f10d9a", "route_table_id": "rtb-068151d6d7a0124bf", "cidr_block": "10.0.10.0/24", "subnet_id": "subnet-07155b2f65e308bdc", "sg_group": "sg-0736e9ea2c048d37d"}, "ca-central-1": {"vpc_id": "vpc-057c6d0dfc55fc466", "route_table_id": "rtb-03b2cd26aa8536fff", "cidr_block": "10.0.11.0/24", "subnet_id": "subnet-0f68b073ed9bfebce", "sg_group": "sg-0293f361fd776fc07"}, "eu-central-1": {"vpc_id": "vpc-064550f1e2113a8a2", "route_table_id": "rtb-026121daac5d9c33c", "cidr_block": "10.0.12.0/24", "subnet_id": "subnet-048aceb7122ffeec6", "sg_group": "sg-0e1ac28986012fb98"}, "eu-west-1": {"vpc_id": "vpc-04865e2af67c3ab4c", "route_table_id": "rtb-06667c34e943d2f44", "cidr_block": "10.0.13.0/24", "subnet_id": "subnet-0f43dd376b591278d", "sg_group": "sg-01e01850d3e818ee5"}, "eu-west-2": {"vpc_id": "vpc-09a94ea04352ad92d", "route_table_id": "rtb-030fa4486f0da1f57", "cidr_block": "10.0.14.0/24", "subnet_id": "subnet-0b13fc62f6692f869", "sg_group": "sg-05cfea0246cf499fc"}, "eu-west-3": {"vpc_id": "vpc-0967b20f439089e5c", "route_table_id": "rtb-07a3e94f666e55b21", "cidr_block": "10.0.15.0/24", "subnet_id": "subnet-059568399a182272d", "sg_group": "sg-0630e085829dbfacc"}, "eu-north-1": {"vpc_id": "vpc-02b331f495c905cc2", "route_table_id": "rtb-0ecc124842edb1ab4", "cidr_block": "10.0.16.0/24", "subnet_id": "subnet-06b02bb4ff0a3a0d1", "sg_group": "sg-09da4927f987329ec"}, "sa-east-1": {"vpc_id": "vpc-0a92d3c9112fe2898", "route_table_id": "rtb-0d466eec3d090c712", "cidr_block": "10.0.17.0/24", "subnet_id": "subnet-0173cd32fc83f09d2", "sg_group": "sg-0cb0401614b6f4394"}}''')

Please copy the code from "Coordinator.py" and "Script.py" from the corresponding folder to the following "script_content_coordinator" and "script_content_client" respectively.

In [3]:
script_content_coordinator= '''import socket
import time
import threading
import pickle
import queue
import argparse
import random
import psutil
import sys
import os
import signal

from collections import defaultdict
from flask import Flask, jsonify, render_template, request
CHUNK_SIZE = 1024
app = Flask(__name__)
logs = []
logs_simplified = []
logs_lock=threading.Lock()
Shards_=[]
f_=[]
BB_=[]
class Coordinator:
    def __init__(self, address, total_nodes, f, BB, num_shards, repeat_times):
        self.address = address
        self.total_nodes = total_nodes
        self.num_shards = num_shards
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.bind(self.address)
        self.server_socket.listen(14)
        self.shards = defaultdict(list)
        self.repeat_times = 0
        self.shard_leaders =[]
        self.node_addresses = []
        self.node_address_lock = threading.Lock()
        self.finished_times = []
        self.finished_times_lock = threading.Lock()
        self.finished_ = defaultdict(set)
        self.finished_lock = threading.Lock()
        self.messages_to_send=defaultdict(list)
        self.meesages_to_send_lock= threading.Lock()
        self.repeat_times = 0
        self.rpt= repeat_times
        self.message_queue = queue.Queue()
        self.broadcast_queue =queue.Queue()
        self.f = f
        self.BB= BB
        self.closed=False
        self.finish_register=False
        threading.Thread(target=self.listen_to_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.process_messages).start()
        threading.Thread(target=self.broadcasting_).start()
        threading.Thread(target=self.broadcasting_).start()
    def listen_to_messages(self):
        try:
            while not self.closed:
                client_socket, _ = self.server_socket.accept()
                threading.Thread(target=self.process_data,args=(client_socket,)).start()
        except Exception as e:
            print("error!!!!!!!!!!!!",e)
    def process_data(self, client_socket):
        try:
            data = b''
            client_socket.settimeout(3.0)  # Adjust timeout as necessary
            while True:
                chunk = client_socket.recv(CHUNK_SIZE)
                if not chunk:  # No more data, client closed connection
                    break
                data += chunk
                if b"*EOF" in data:  # Check for EOF marker
                    client_socket.sendall(b"pong")  # Acknowledge EOF
                    data = data.split(b"*EOF")[0]  # Remove EOF marker and extra data if any
                    message = pickle.loads(data)  # Deserialize data
                    if isinstance(message,list):
                        for u in message:
                            self.message_queue.put((u, ))
                    else:
                        self.message_queue.put((message, ))  # Enqueue message
                    break  # Exit the loop after handling EOF
        except socket.timeout:
            print("Socket timed out")
        except pickle.PickleError:
            print("Failed to deserialize data")
        except Exception as e:
            print("Error:", e)
        finally:
            client_socket.close()  # Ensure socket is closed
        
    def assign_nodes_to_shards(self):
        shard_size = self.total_nodes // self.num_shards
        extra_nodes = self.total_nodes % self.num_shards
        start_index = 0
        for i in range(self.num_shards):
            end_index = start_index + shard_size + (1 if i < extra_nodes else 0)
            self.shards[i] = self.node_addresses[start_index:end_index]
            start_index = end_index

    def process_messages(self):
        while True:
            try:
                message,  = self.message_queue.get()
                if self.finish_register and message['address'] not in self.node_addresses:
                    continue
                if message['type'] == 'register':
                  #  mes_ = {
                  #      'type': 'ack',
                  #      'seq_num': 'register'
                  #  }
                    if len(self.node_addresses) == self.total_nodes:
                        continue
                   # with self.meesages_to_send_lock:
                   #     if message['address'] in self.messages_to_send:
                   #         self.messages_to_send[message['address']].append(mes_)
                   #     else:
                   #         self.messages_to_send[message['address']]= [mes_]
                   #     self.broadcast_queue.put((message['address']))
                    if message['address'] in self.node_addresses:
                        continue
                    with self.node_address_lock:
                        self.node_addresses.append(message['address'])
                        log_message = f"Node {message['node_id']} registered with address {message['address']}"
                        print(log_message)
                        with logs_lock:
                            logs.append(log_message)
                            logs_simplified.append(log_message)
                        if len(self.node_addresses) == self.total_nodes:
                            self.node_addresses=random.sample(self.node_addresses, len(self.node_addresses))
                            threading.Thread(target=self.broadcast_node_addresses).start()
                            self.finish_register =True
                            print ("start to broadcast")
                elif message['type'] == 'finished':
                    #mes_ = {
                    #    'type': 'ack',
                    #    'seq_num': message['seq_num']
                    #}
                    #print ("finished message", message)
                    #with self.meesages_to_send_lock:
                    #    if message['address'] in self.messages_to_send:
                    #        self.messages_to_send[message['address']].append(mes_)
                    #    else:
                    #        self.messages_to_send[message['address']]= [mes_]
                    #    self.broadcast_queue.put((message['address']))
                    with self.finished_lock:
                        if message['node_id'] in self.finished_[message['seq_num']]:
                            continue
                    self.finished_[message['seq_num']].add(message['node_id'])
                    log_message = f"Node {message['node_id']} finished at {message['finish_time']}"
                    print(log_message)
                    with logs_lock:
                        logs.append(log_message)
                    with self.finished_times_lock:
                        self.finished_times.append(message['finish_time'])
                    if len(self.finished_times) == self.total_nodes:
                        last_finish_time = max(self.finished_times)
                        log_message = f"{self.repeat_times} The last node finished at {last_finish_time}"
                        print(log_message)
                        with logs_lock:
                            logs.append(log_message)
                            logs.append(self.repeat_times)
                            logs_simplified.append(log_message)
                            logs_simplified.append(self.repeat_times)
                        if self.repeat_times<self.rpt:
                            self.repeat_times+=1
                            with logs_lock:
                                log_message = "Repeat request sent"
                                logs.append(log_message)
                            threading.Thread(target=self.request_repeat_experiment, args=(self.repeat_times,)).start()
                            with logs_lock:
                                log_message = "Repeat request done"
                                logs.append(log_message)
                        else:
                            self.request_start_a_new_experiment()
            except Exception as e: 
                log_message = f"Something was wrong with the thread {e}, {message}"
                with logs_lock:
                    logs.append(log_message)
                print (log_message)
            finally:
                self.message_queue.task_done()

    def broadcast_node_addresses(self):
        with logs_lock:
            log_message = "Broadcast address started"
            logs.append(log_message)        
        self.assign_nodes_to_shards()
        for shard_id, addresses in self.shards.items():
            message = {
                'type': 'node_addresses',
                'addresses': addresses,
                'shard_id': shard_id,
                'f':self.f,
                'BB':self.BB
            }
            self.shard_leaders.append(addresses[0])
            self.send_message(addresses[0], message)
        log_message = "Broadcast address finished"
        with logs_lock:
            logs.append(log_message)
        self.request_repeat_experiment(0)
        log_message = "Experiment requested"
        with logs_lock:
            logs.append(log_message)

    def send_message(self, address, message):
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(address)
        data = pickle.dumps(message)
        client_socket.sendall(data+b"*EOF")
        client_socket.settimeout(3.0)
        try:
            data=client_socket.recv(1024)
            client_socket.close()
            if data.decode('utf-8') != "pong":
                raise Exception ('no pong')
        except:
            with self.meesages_to_send_lock:
                if address in self.messages_to_send:
                    self.messages_to_send[address].append(message)
                else:
                    self.messages_to_send[address]= [message]
                self.broadcast_queue.put((address))
    
    def broadcasting_(self):
        while True:
            try:
                address = self.broadcast_queue.get()
                with self.meesages_to_send_lock:
                    temp=self.messages_to_send[address]
                    if temp != []:
                        self.messages_to_send[address]= []
                if temp != []:
                    self.send_message(address,temp)
            except Exception as e:
                if self.finished:
                    break
                print ("EError:",e)
    
    def request_repeat_experiment(self,ID):
        time.sleep(20)
        message = {
            'type': 'repeat_experiment',
            'start_time' : time.time(),
            'ID':ID
        }
        self.finished_times=[]
        self.finished_= defaultdict(set)
        for address in self.shard_leaders:
            self.send_message(address, message)

    def request_start_a_new_experiment(self):
        count_=0
        while (not self.message_queue.empty()) or  (not self.broadcast_queue.empty()):
            time.sleep(5)
        print ("request a new experiment")
        message = {
            'type': 'start_new_experiment'
        }
        global Shards_
        if len(Shards_) == 1:
            self.closed=True
            self.server_socket.close()
        else:
            self.finish_register =False
            self.node_addresses = []
            self.restart_program()
            ttk=self.shard_leaders
            self.shard_leaders=[]
            for address in ttk:
                self.send_message(address, message)
    def send_ack_message(self, address,log_message):
        message = {
            'type': 'ack',
            'log': log_message
        }
        self.send_message(address, message)
    def restart_program(self):
        global Shards_
        global f_
        if len(Shards_) == 1:
            return 
        else:
            f_=f_[1:]
            Shards_=Shards_[1:]
            self.num_shards = Shards_[0]
            self.shards = defaultdict(list)
            self.repeat_times = 0
            self.finished_times = []
            self.finished_ = defaultdict(set)
            self.messages_to_send=defaultdict(list)
            self.f = f_[0]
            self.BB = BB_[0]
@app.route('/logs', methods=['GET'])
def get_logs():
    return jsonify(logs)
@app.route('/logs_simplified', methods=['GET'])
def get_logs_simplified():
    return jsonify(logs_simplified)
@app.route('/exit___', methods=['GET'])
def exit__():
    self.server_socket.close()
    os.kill(os.getpid(), signal.SIGTERM)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Run the coordinator node.")
    parser.add_argument('--host', type=str, required=True, help="Coordinator host address")
    parser.add_argument('--port', type=int, required=True, help="Coordinator port number")
    parser.add_argument('--total_nodes', type=int, required=True, help="Total number of nodes")
    parser.add_argument('--num_shards', type=str, required=True, help="Number of shards")
    parser.add_argument('--f', type=str, required=True, help="Fraction of adversary")
    parser.add_argument('--bb', type=str, required=True, help="If using Byzantine Reliable broadcast")
    parser.add_argument('--repeat_times', type=int, required=True, help="repeat_times")
    parser.add_argument('--web_port',type=int,required=True, help="web port")
    args = parser.parse_args()
    address = (args.host, args.port)
    total_nodes = args.total_nodes
    num_shards = eval(args.num_shards)
    BB = eval(args.bb)
    f =  eval(args.f)
    try:
        global global_coordinator
        global_coordinator = Coordinator(address, total_nodes, f[0], BB[0], num_shards[0], args.repeat_times)
        Shards_ = num_shards
        f_ = f
        BB_= BB
        app.run(host='0.0.0.0', port=args.web_port)
    except Exception as e:
        print (f"Error!!! {e}")
'''


script_content_client='''import socket
import threading
import pickle
import random
import time
import hashlib
import math
import queue
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict
from collections import deque
import base64
import argparse
CHUNK_SIZE = 1024 
RETRY_LIMIT = 15 
RETRY_DELAY = 2   
global_instance_list = []

def hash_content(content):
    return hashlib.sha256(content.encode()).hexdigest()
class Node:
    def __init__(self, node_id, address, coordinator_address):
        self.node_id = node_id
        self.address = address
        self.shard_addresses = []  # List of addresses within the same shard
        self.shard_id = None
        self.is_primary = False
        self.coordinator_address = coordinator_address
        self.state = defaultdict(set)
        self.view = 0
        self.f = 0
        self.pre_prepared = defaultdict(set)
        self.pre_prepared_lock = threading.Lock()
        self.prepared = defaultdict(set)
        self.prepared_lock = threading.Lock()
        self.committed = defaultdict(set)
        self.committed_lock = threading.Lock()
        self.received_messages = defaultdict(dict)
        self.received_messages_lock  = threading.Lock()
        self.broadcast_status = defaultdict(lambda: defaultdict(set))
        self.broadcast_status_lock  = threading.Lock()
        self.start_time = defaultdict(lambda: defaultdict(set))
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.message_queue = queue.Queue()
        self.received_message_queue = queue.Queue()
        self.messages_to_send=defaultdict(list)
        self.meesages_to_send_lock=threading.Lock()
        self.qualified = defaultdict(int)
        self.qualified_lock = threading.Lock()
        self.ack_list = []
        self.ack_list_lock=threading.Lock()
        self.retransit_sent =  defaultdict(list)
        self.retransit_sent_lock=threading.Lock()
        self.received = defaultdict(bool)
        self.received_lock = threading.Lock()
        self.finished = False
        self.finished_completed = False
        self.BB=False
        try:
            self.server_socket.bind(self.address)
            print(f"Successfully bound to {self.address}")
        except socket.error as e:
            print(f"Failed to bind to {self.address}: {e}")
            # Bind error occurred, let's try binding to a random port
            while True:
                # Generate a random port number
                random_port = random.randint(1024, 65535)
                try:
                    # Attempt to bind to the new port
                    self.server_socket.bind((self.address[0], random_port))
                    print(f"Successfully bound to {(self.address[0], random_port)}")
                    self.address= (self.address[0], random_port)
                    break  # Exit the loop if successful
                except socket.error as e:
                    print(f"Failed to bind to {(self.address[0], random_port)}: {e}")
                    continue  # Continue trying with another random port
        self.server_socket.listen(5)
        threading.Thread(target=self.listen_to_messages).start()
        threading.Thread(target=self.broadcasting_).start()
        threading.Thread(target=self.broadcasting_).start()
        threading.Thread(target=self.message_process).start()
        threading.Thread(target=self.await_broadcast).start()
        self.register_with_coordinator()
    def if_finished (self):
        return self.finished
    def finish (self):
        self.finished_completed=True
        self.server_socket.close()
    def register_with_coordinator(self):
        message = {
            'type': 'register',
            'node_id': self.node_id,
            'address': self.address
        }
        self.send_message(self.coordinator_address, message)
    def listen_to_messages(self):
        while not self.finished_completed:
            client_socket, _ = self.server_socket.accept()
            threading.Thread(target=self.handle_message, args=(client_socket,)).start()

    def handle_message(self,client_socket):
        data = b''
        client_socket.settimeout(3.0)  # Adjust timeout as necessary
        try:
            while not self.finished_completed:
                chunk = client_socket.recv(CHUNK_SIZE)
                if not chunk:
                    break
                data += chunk
                if b"*EOF" in data:
                    client_socket.sendall(b"pong")
                    break
        except Exception as e:
            return
        finally:
            client_socket.close()
        message_=pickle.loads(data.split(b"*EOF")[0])
        if isinstance(message_,dict):
            message_=[message_]
        if isinstance(message_,list):
            for message in message_:
                if message['type'] == 'node_addresses':
                        self.shard_addresses = message['addresses']
                        self.shard_id = message['shard_id']
                        self.BB = message['BB']
                        print (self.shard_addresses)
                        self.is_primary = (self.address == self.shard_addresses[0])  # First node in list is primary
                        temp= message.get('retransmit')
                        if len(global_instance_list)>0:
                            try:
                                for ii in global_instance_list:
                                    if ii.if_finished():
                                        ii.finish()
                                        global_instance_list.remove (ii)
                            except Exception as e:
                                print (f"error in delecting {e}")
                                pass
                        if temp!=None and temp >0:
                            self.retransmit(message)
                        else:
                            if self.is_primary and temp == None:
                                print ("I am primary")
                                message['retransmit']=2
                                self.direct_broadcast(message)
                            self.f =int((len(self.shard_addresses)-1)/int(message['f']))
                else:
                    if self.f!=0:
                        self.receive_message(message)
                    else:
                        self.received_message_queue.put(message)
    def message_process(self):
        while not self.finished_completed:
            if self.f!=0: #implying that the node address are received.
                message = self.received_message_queue.get()
                try:
                    self.receive_message(message)
                    if self.received_message_queue.empty():
                        break
                except Exception as e:
                    print(f"There is a error in {e}")
    def broadcasting_(self):
        while not self.finished_completed:
            try:
                address = self.message_queue.get()
                with self.meesages_to_send_lock:
                    temp=self.messages_to_send[address]
                    if len(temp) != 0:
                        self.messages_to_send[address]= []
                        self.send_message(address,temp)
            except Exception as e:
                print ("EError:",e)
    def send_message(self, address, message):
        try:
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect(address)
            data = pickle.dumps(message)
            client_socket.sendall(data+b"*EOF")
            client_socket.settimeout(3.0)
            data=client_socket.recv(1024)
            client_socket.close()
            if data.decode('utf-8') != "pong":
                raise Exception ('no pong')
        except:
            with self.meesages_to_send_lock:
                if isinstance(message,list):
                    self.messages_to_send[address]+=message
                else:
                    self.messages_to_send[address].append(message)
                self.message_queue.put((address))
        #sender(address, message)
    def direct_broadcast(self, message):
        my_index = self.shard_addresses.index(self.address)
        sqrt_len = math.ceil(math.sqrt(len(self.shard_addresses)))
        with self.meesages_to_send_lock:
            for i in range (sqrt_len):
                next_index = (my_index +  (i) * sqrt_len ) % len(self.shard_addresses)
                self.messages_to_send[self.shard_addresses[next_index]].append(message)
                self.message_queue.put((self.shard_addresses[next_index]))
                
    def retransmit(self, message):
 #       # Get the index of this node in the shard_addresses
        if  message['retransmit']==0:
            return
        my_index = self.shard_addresses.index(self.address)
        sqrt_len = math.ceil(math.sqrt(math.sqrt(len(self.shard_addresses))))
        hash_content_=hash_content(str(message))
        if message['retransmit']==2:
            message['retransmit']=1
            with self.meesages_to_send_lock:
                if hash_content_ not in self.retransit_sent["2"]:
                    self.retransit_sent["2"].append(hash_content_)
                    for i in range (sqrt_len):
                        next_index = (my_index + (i) * sqrt_len) % len(self.shard_addresses)
                        self.messages_to_send[self.shard_addresses[next_index]].append(message)
                        self.message_queue.put((self.shard_addresses[next_index]))
         #   self.retransmit(message)
        elif message['retransmit']==1:
            message['retransmit']=0
            with self.meesages_to_send_lock:
                if hash_content_ not in self.retransit_sent["1"]:
                    self.retransit_sent["1"].append(hash_content_)
                    for i in range (sqrt_len):
                        next_index = (my_index + i) % len(self.shard_addresses)
                        self.messages_to_send[self.shard_addresses[next_index]].append(message)
                        self.message_queue.put((self.shard_addresses[next_index]))

    def receive_message(self, message):
        temp= message.get('retransmit')
        if temp!=None and temp >0:
            self.retransmit(message)
        else:
            if message['type'] == 'repeat_experiment':
                print(f"Node {self.node_id} received request to repeat experiment.")
                if message['ID'] not in self.start_time:
                    self.start_time[message['ID']][self.view]=message['start_time']
                    self.state[message['ID']] = 'initial'
                if self.is_primary and temp == None:
                    print ("I am primary")
                    message['retransmit']=2
                    self.direct_broadcast(message)
                    self.start_protocol("Sample Request" * 1024 * 5,message['ID'])
            elif message['type'] == 'start_new_experiment':
                print(f"Node {self.node_id} received request to start new experiment.")
                if self.is_primary and temp == None:
                    print ("I am primary")
                    message['retransmit']=2
                    self.direct_broadcast(message)
                if temp ==0 and  self.finished == False:
                    self.finished = True
                    self.activate_a_new_instance(self.node_id, self.address, self.coordinator_address)
            else:
                message_type = message['type']
                if message_type == 'ack-votes':
                    if hash_content(str(message)) not in self.received:
                        with self.received_lock:
                            self.received[hash_content(str(message))]=True
                        for tt in message['message_id']:
                            with self.qualified_lock:
                                if tt not in self.qualified:
                                    self.qualified[tt] = 1
                                else:
                                    self.qualified[tt]+=1
                            if self.qualified[tt]>self.f and tt in self.received_messages:
                                with self.qualified_lock:
                                    self.qualified[tt]=-5000
                                self.handle_qualified(self.received_messages[tt])
                else:
                    if self.finished ==True:
                        return
                    if self.BB==False:
                        self.handle_qualified(message)
                    else:
                        message_id = (hash_content(str(message)))
                        if message_id not in self.received_messages:
                            with self.received_messages_lock:
                                self.received_messages[message_id]= message
                            with self.ack_list_lock:
                                self.ack_list.append(message_id)
                            if self.qualified[message_id]>self.f:
                                self.handle_qualified(message)
                                with self.qualified_lock:
                                    self.qualified[message_id]=-5000 #threading.Thread(target=self.handle_qualified, args=(message,)).start()
    def await_broadcast (self):
        while not self.finished_completed:
            time.sleep(5)
            with self.ack_list_lock:
                if len(self.ack_list)!=0:
                    message={'message_id':self.ack_list, 'type':"ack-votes", 'sender':self.node_id, 'view':self.view, 'retransmit':2}
                    self.direct_broadcast(message)
                    self.ack_list= []
    def handle_qualified (self, message):
                message_type = message['type']
                view = message['view']
                seq_num = message['seq_num']
                content = message.get('content')
                content_hash = message.get('content_hash')
                if content_hash is None and content is not None:
                    content_hash = hash_content(content)
                sender = message['sender']
                message_id = (message_type, view, seq_num, content_hash)
                message_type = message['type']
                if message_type == 'pre-prepare':
                    if view == self.view and self.valid_message(message):
                        if message_id not in self.pre_prepared[seq_num]:
                            with self.pre_prepared_lock:
                                self.pre_prepared[seq_num].add(message_id)
                            if 'prepare' not in self.broadcast_status[seq_num][view]:
                                with self.broadcast_status_lock:
                                    self.broadcast_status[seq_num][view].add('prepare')
                                self.direct_broadcast({
                                    'type': 'prepare',
                                    'view': view,
                                    'seq_num': seq_num,
                                    'content_hash': content_hash,
                                    'sender': self.node_id,
                                    'retransmit' : 2
                                })
                elif message_type == 'prepare':
                    if view == self.view and self.valid_message(message):
                        prepare_id = (sender, seq_num, content_hash)
                        if prepare_id not in self.prepared[seq_num]:
                            with self.prepared_lock:
                                self.prepared[seq_num].add(prepare_id)
                            if len(self.prepared[seq_num]) >len(self.shard_addresses)-self.f and 'commit' not in self.broadcast_status[seq_num][view]:
                                with self.broadcast_status_lock:
                                    self.broadcast_status[seq_num][view].add('commit')
                                self.direct_broadcast({
                                    'type': 'commit',
                                    'view': view,
                                    'seq_num': seq_num,
                                    'content_hash': content_hash,
                                    'sender': self.node_id,
                                    'retransmit' : 2
                                })
                elif message_type == 'commit':
                    if view == self.view and self.valid_message(message):
                        commit_id = (sender, seq_num, content_hash)
                        if commit_id not in self.committed[seq_num]:
                            with self.committed_lock:
                                self.committed[seq_num].add(commit_id)
                            if len(self.committed[seq_num]) > len(self.shard_addresses)-self.f and 'terminate' not in self.broadcast_status[seq_num][view]:
                                with self.broadcast_status_lock:
                                    self.broadcast_status[seq_num][view].add('terminate')
                if 'prepare' in self.broadcast_status[seq_num][view] and 'commit' in self.broadcast_status[seq_num][view] and 'terminate' in self.broadcast_status[seq_num][view]:
                        self.execute(seq_num)
    def valid_message(self, message):
        required_keys = {'type', 'view', 'seq_num', 'sender'}
        return all(key in message for key in required_keys)
    def execute(self, seq_num):
        with self.broadcast_status_lock:
            if self.state[seq_num] == 'initial':
                self.state[seq_num] = 'executed'
                finish_time = time.time()
                print(f"Node {self.node_id} finished execution at {finish_time}")
                self.send_finished_message(finish_time,seq_num)
    def send_finished_message(self, finish_time,seq_num):
        message = {
            'type': 'finished',
            'seq_num':seq_num,
            'node_id': self.node_id,
            'address': self.address,
            'finish_time': finish_time - self.start_time[seq_num][self.view]
        }
        self.send_message(self.coordinator_address, message)
        print ("I sent the final signal to coordinator")
    def start_protocol(self, content,ID):
        print(f"Protocol started at { time.time()}")
        seq_num = ID
        content_hash = hash_content(content)
        self.state[seq_num] = 'initial'  # Reset state
        self.direct_broadcast({
            'type': 'pre-prepare',
            'view': self.view,
            'seq_num': seq_num,
            'content': content,
            'content_hash': content_hash,
            'sender': self.node_id,
            'retransmit' : 2
        })
    def activate_a_new_instance (self,node_id, address, coordinator_address):
        global global_instance_list
        global_instance_list.append(Node(node_id, address, coordinator_address))
        print("started a new run")
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Run a PBFT node.")
    parser.add_argument("--node_id", type=int, required=True, help="Node ID")
    parser.add_argument("--host", type=str, required=True, help="Host address")
    parser.add_argument("--port", type=int, required=True, help="Port number")
    parser.add_argument("--coordinator_address", type=str, required=True, help="Coordinator node address")
    args = parser.parse_args()
    node_id = args.node_id
    address = (args.host, args.port)
    coordinator_address = eval(args.coordinator_address)
    global_instance_list.append(Node(node_id, address, coordinator_address))
    print("started")'''

def start_node(node_id, host, port, coordinator_address):
    cmd = f'python3 /tmp/script.py --node_id {str(node_id)} --host {host} --port {str(port)}  --coordinator_address "{str(coordinator_address)}"'
    return cmd
def escape_single_quotes(input_string):
    # Replace each single quote with an escaped single quote
    return input_string.replace("'", "\\'")
def replace_single_with_double_quotes(input_string):
    # Replace each single quote with a double quote
    return input_string.replace("'", '"')

ami_ids = {}

# Get AMI IDs for each region
for region in regions:
    ami_id = get_amazon_linux_2023_ami(region)
    ami_ids[region] = ami_id
    print(f"Region: {region}, AMI ID: {ami_id}")
import random
import ipaddress
def generate_random_ip(subnet):
    # Convert the subnet to an ip_network object
    subnet=subnet[:-1]+"5" # we used 25 instead of 24 
    network = ipaddress.ip_network(subnet, strict=False)
    
    # Get all valid host addresses within the subnet
    hosts = list(network.hosts())
    
    # Exclude the first four and the last IP address in the subnet
    valid_hosts = hosts[4:-1]
    
    # Generate a random IP address from the remaining valid hosts
    random_ip = random.choice(valid_hosts)
    
    return str(random_ip)

Region: us-east-1, AMI ID: ami-07fe1a32c66493b73
Region: us-east-2, AMI ID: ami-0bfa2fb693fc2aeb2
Region: us-west-1, AMI ID: ami-08b5147d32395e196
Region: us-west-2, AMI ID: ami-003ef79521859c6a4
Region: ap-south-1, AMI ID: ami-01d2dc252ecee8dc6
Region: ap-northeast-3, AMI ID: ami-05c1e54e8cc8e2ca4
Region: ap-northeast-2, AMI ID: ami-0d412c1f9bec3c853
Region: ap-southeast-1, AMI ID: ami-0a4dd55192537750e
Region: ap-southeast-2, AMI ID: ami-073e18c6c2b1d13f2
Region: ap-northeast-1, AMI ID: ami-0a0d112f5f73bd1f2
Region: ca-central-1, AMI ID: ami-0d8ee690ffbc68be6
Region: eu-central-1, AMI ID: ami-06d557929ce0f69b8
Region: eu-west-1, AMI ID: ami-0b393e70c529ce632
Region: eu-west-2, AMI ID: ami-0f3246b739bd4209e
Region: eu-west-3, AMI ID: ami-0fd0cd7e696f11944
Region: eu-north-1, AMI ID: ami-02383ef4ced32b1bd
Region: sa-east-1, AMI ID: ami-0b07744f9701122c1


# Run the experiment:

In [11]:
import boto3
import random
import logging
import threading
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
from collections import defaultdict
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
Assigned_ip=[]
all_instances=defaultdict(list)
Failed_instances = []
kk = threading.Lock()
def run_remote_script(instance_id, command, aws_region='us-east-1'):
    count=0
    while True:
        try:
            ssm_client = boto3.client('ssm', region_name=aws_region)
            response = ssm_client.send_command(
                InstanceIds=[instance_id],
                DocumentName='AWS-RunShellScript',
                Parameters={'commands': [command]},
            TimeoutSeconds=172800)
            command_id = response['Command']['CommandId']
            return
        except Exception as e:
            count+=1
            print(f"Failed to send command: {e}")
            if count>=5:
                if {'id':instance_id,'command':command,'region':aws_region} not in Failed_instances:
                    Failed_instances.append({'id':instance_id,'command':command,'region':aws_region})
                break
            time.sleep(5)

def create_instances( total_nodes, port, REGIONS, node_count, shard_counts, fs,BB,nodes_per_instance,rpt):
    instances_ = []
    coordinator_address = ('0.0.0.0', 3000)  # Address of the coordinator node
    count_=0
    print (f"There are {len(REGIONS)} regions.")
    total_nodes_=node_count
    with kk:
        for k, region in enumerate(REGIONS):
            try:
                print (f"start on {region}")
                instances_1 = []
                for i in range(int(total_nodes / nodes_per_instance/ len(REGIONS)) + (int(total_nodes /nodes_per_instance) % len(REGIONS) +1 if k == 0 else 0)):
                    if total_nodes_<=0:
                        break
                    ec2 = boto3.resource('ec2', region_name=region)
                    ec2_c=boto3.client('ec2', region_name=region)
                    if k==0 and i==0:# coordinator
                        coordinator_IP=generate_random_ip(vpc_info[region]['cidr_block'])
                        Assigned_ip.append(coordinator_IP)
                        print ("Coordinator IP:", coordinator_IP)
                        coordinator_address = (coordinator_IP, 3000)
                        userdata='''#!/bin/bash
                sudo yum install -y python3
                sudo yum install -y python3-pip
                pip install Flask
                pip install psutil
               '''+f"echo $'{escape_single_quotes(script_content_coordinator)}' > /tmp/coordinator.py \n nohup python3 /tmp/coordinator.py --host 0.0.0.0 --port 3000 --total_nodes {node_count} --num_shards '{str(shard_counts)}' --f '{str(fs)}' --bb '{str(BB)}' --repeat_times {rpt} --web_port 5000 >/tmp/output.txt 2>&1 &"
                        try:
                            instance = ec2.create_instances(
                                ImageId=ami_ids[region],  # Replace with a valid AMI ID
                                InstanceType='t3.micro',
                                MinCount=1,
                                MaxCount=1,
                                NetworkInterfaces=[{
                                    'DeviceIndex': 0,
                                    'AssociatePublicIpAddress': True,
                                    'SubnetId': vpc_info[region]['subnet_id'],  # Replace with a valid Subnet ID
                                    'Groups': [vpc_info[region]["sg_group"]],
                                    'PrivateIpAddress':coordinator_IP
                                }],
                                IamInstanceProfile={
                                    'Name': 'Experiment_'  # Replace with the name of your IAM role
                                },
                            UserData='''#!/bin/bash
                            sudo yum install -y amazon-ssm-agent
    sudo systemctl enable amazon-ssm-agent
    sudo systemctl start amazon-ssm-agent''')[0]
                        except:
                            instance = ec2.create_instances(
                                ImageId=ami_ids[region],  # Replace with a valid AMI ID
                                InstanceType='t2.micro',
                                MinCount=1,
                                MaxCount=1,
                                NetworkInterfaces=[{
                                    'DeviceIndex': 0,
                                    'AssociatePublicIpAddress': True,
                                    'SubnetId': vpc_info[region]['subnet_id'],  # Replace with a valid Subnet ID
                                    'Groups': [vpc_info[region]["sg_group"]],
                                    'PrivateIpAddress':coordinator_IP
                                }],
                                IamInstanceProfile={
                                    'Name': 'Experiment_'  # Replace with the name of your IAM role
                                },
                            UserData='''#!/bin/bash
                            sudo yum install -y amazon-ssm-agent
    sudo systemctl enable amazon-ssm-agent
    sudo systemctl start amazon-ssm-agent''')[0]
                        instances_.append({'instance':instance,'region':region,'UserData': userdata})
                        instances_1.append({'instance':instance,'region':region,'UserData': userdata})
                        all_instances[region].append({'instance':instance,'region':region,'UserData': userdata})
                        logger.info(f"Created instance {instance.id} in region {region}")
                        response = ec2_c.describe_instances(InstanceIds=[instance.id])
                        public_ip = None
                        for reservation in response['Reservations']:
                            for instance in reservation['Instances']:
                                if 'PublicIpAddress' in instance:
                                    public_ip = instance['PublicIpAddress']
                                    break
                        print(f"Accesss the result via:{public_ip}:5000/logs")

                    else:
                        new_ip=generate_random_ip(vpc_info[region]['cidr_block'])
                        while (new_ip in Assigned_ip):
                            new_ip=generate_random_ip(vpc_info[region]['cidr_block'])
                        Assigned_ip.append(new_ip)
                        temp_data=""
                        for er in range (nodes_per_instance):
                            temp_data+="nohup "+ start_node(count_+er, new_ip, port+er, coordinator_address)+" &\n"
                        userdata='''#!/bin/bash
                sudo yum install -y python3
                sudo yum install -y python3-pip
               '''+ f"echo $'{escape_single_quotes(script_content_client)}' > /tmp/script.py \n nohup "+ temp_data
                        try:
                            instance = ec2.create_instances(
                                ImageId=ami_ids[region],  # Replace with a valid AMI ID
                                InstanceType='t3.micro',
                                MinCount=1,
                                MaxCount=1,
                                NetworkInterfaces=[{
                                    'DeviceIndex': 0,
                                    'AssociatePublicIpAddress': True,
                                    'SubnetId': vpc_info[region]['subnet_id'],  # Replace with a valid Subnet ID
                                    'Groups': [vpc_info[region]["sg_group"]],
                                    'PrivateIpAddress':new_ip
                                }],
                                IamInstanceProfile={
                                    'Name': 'Experiment_'  # Replace with the name of your IAM role
                                },
                            UserData='''#!/bin/bash
                            sudo yum install -y amazon-ssm-agent
    sudo systemctl enable amazon-ssm-agent
    sudo systemctl start amazon-ssm-agent''')[0]
                        except:
                            instance = ec2.create_instances(
                                ImageId=ami_ids[region],  # Replace with a valid AMI ID
                                InstanceType='t2.micro',
                                MinCount=1,
                                MaxCount=1,
                                NetworkInterfaces=[{
                                    'DeviceIndex': 0,
                                    'AssociatePublicIpAddress': True,
                                    'SubnetId': vpc_info[region]['subnet_id'],  # Replace with a valid Subnet ID
                                    'Groups': [vpc_info[region]["sg_group"]],
                                    'PrivateIpAddress':new_ip
                                }],
                                IamInstanceProfile={
                                    'Name': 'Experiment_'  # Replace with the name of your IAM role
                                },
                            UserData='''#!/bin/bash
                            sudo yum install -y amazon-ssm-agent
    sudo systemctl enable amazon-ssm-agent
    sudo systemctl start amazon-ssm-agent''')[0]
                        count_+=nodes_per_instance
                        total_nodes_-=nodes_per_instance
                        instances_.append({'instance':instance,'region':region,'UserData': userdata})
                        instances_1.append({'instance':instance,'region':region,'UserData': userdata})
                        all_instances[region].append({'instance':instance,'region':region,'UserData': userdata})
                        logger.info(f"Created instance {instance.id} in region {region}")
                print (f"finished one region: {region}")
                threading.Thread(target=ex,args=(instances_1,region)).start()
                continue
            except (NoCredentialsError, PartialCredentialsError) as e:
                logger.info(f"Error: {str(e)}")
                continue
        return instances_

def ex(instance_,region):
    
    wait_for_instance_initialization([instance['instance'].id for instance in instance_], region)
    for instance in instance_:
        run_remote_script(instance['instance'].id, instance['UserData'], instance['region'])    
    
    

def wait_for_instance_initialization(instance_ids, region):
    with kk:
        try:
            ec2_client = boto3.client('ec2', region_name=region)
            waiter = ec2_client.get_waiter('instance_status_ok')
            logger.info(f"Waiting for instances {instance_ids} to be in 'running' state and initialized")
            waiter.wait(InstanceIds=instance_ids)
            logger.info(f"Instances {instance_ids} are ready")
        except ClientError as e:  # Corrected the exception type
            logger.error(f"Error while waiting for instances to be ready: {str(e)}")
            return False
        except Exception as e:  # General catch for other possible exceptions
            logger.error(f"Unexpected error: {str(e)}")
            return False
        return True


import time            
instances= []
if __name__ == "__main__":
    num_instances = 54*17*6 # Number of nodes
        # Create instances
    global instances
    instances = create_instances(num_instances,35001,regions,5000,[1,1,1,17,17,5],[2,2,3,3,2,3,3],[True,False,True,True,False,False],6,3)
    if not instances:
        logger.error("No instances created. Exiting.")
        exit(1)

There are 17 regions.
start on us-east-1
Coordinator IP: 10.0.1.79


INFO:__main__:Created instance i-03b754fff4f3b3a6b in region us-east-1


Accesss the result via:18.234.218.3:5000/logs


INFO:__main__:Created instance i-002bffed5917332f5 in region us-east-1
INFO:__main__:Created instance i-040afabf3fee46084 in region us-east-1
INFO:__main__:Created instance i-05e415f875eb21e65 in region us-east-1
INFO:__main__:Created instance i-0de10ea498f31e498 in region us-east-1
INFO:__main__:Created instance i-04ad4ea2e163e505d in region us-east-1
INFO:__main__:Created instance i-014bb4ea74358b928 in region us-east-1
INFO:__main__:Created instance i-0f9fe6fbe8ab29be6 in region us-east-1
INFO:__main__:Created instance i-0ecd4311e81d5f85c in region us-east-1
INFO:__main__:Created instance i-09e13b4297f11f059 in region us-east-1
INFO:__main__:Created instance i-084aea33972d518c7 in region us-east-1
INFO:__main__:Created instance i-04318d5b9d9d4cf1c in region us-east-1
INFO:__main__:Created instance i-01468aac027e2ca4e in region us-east-1
INFO:__main__:Created instance i-040cf87134c042149 in region us-east-1
INFO:__main__:Created instance i-033cb90bc1f8179a0 in region us-east-1
INFO:_

finished one region: us-east-1
start on us-east-2


...

INFO:__main__:Waiting for instances ['i-03b754fff4f3b3a6b', 'i-002bffed5917332f5', 'i-040afabf3fee46084', 'i-05e415f875eb21e65', 'i-0de10ea498f31e498', 'i-04ad4ea2e163e505d', 'i-014bb4ea74358b928', 'i-0f9fe6fbe8ab29be6', 'i-0ecd4311e81d5f85c', 'i-09e13b4297f11f059', 'i-084aea33972d518c7', 'i-04318d5b9d9d4cf1c', 'i-01468aac027e2ca4e', 'i-040cf87134c042149', 'i-033cb90bc1f8179a0', 'i-0a32cec27dea9c9d1', 'i-00f0163c9512582fb', 'i-0ed0a64a0dabd06a3', 'i-071eefd6e2a6e6d32', 'i-06f609add969c3437', 'i-0365ab1337ba5322e', 'i-0ba5ca19290d3aa70', 'i-0d350cd8d22183248', 'i-0247088cc4784dd21', 'i-0ac6446938e4d6ea6', 'i-0d15cd53ec48f31aa', 'i-05213ea32c3352e7a', 'i-0d2af3986273f84a0', 'i-0dae69901f60c2398', 'i-010754381e75fecf4', 'i-09104507a785e24ca', 'i-09220fccef1a762db', 'i-09775e089edb85aa7', 'i-02168a38f0308d3a0', 'i-0a71ef1670208e0a6', 'i-06de8fe42c396c4ed', 'i-0b1dadd8c1e64e81b', 'i-02e89890c8ee684f6', 'i-0c633a74e898e0754', 'i-0c6b379f4491ac911', 'i-08ddb91ee8e6099fc', 'i-05eeef5c89847e50a

INFO:__main__:Instances ['i-0b1cd1daffab7e3d2', 'i-06228df9efb67c05e', 'i-0bd252f5f8b67fd70', 'i-05f261df802c43cdf', 'i-03a7431b9d9ca491f', 'i-0baaecacec30b0c2e', 'i-0c6f127fdc4c830be', 'i-07f79dd733e086b2e', 'i-04eb852b3f081abfb', 'i-06b8ed500ef89f734', 'i-0194977f6501faaf8', 'i-0812feb5a64aa4b31', 'i-0320526cacd555791', 'i-0714936dedd26e024', 'i-0070d9586e3c0a9e0', 'i-08c8c0121ad4bd18e', 'i-08d52e03394fa0318', 'i-075ad307822dd04b2', 'i-09daf2e101b9c9577', 'i-0b553531670fb5c03', 'i-0da9bd95d208e7421', 'i-08ce3bf0053bc659d', 'i-0bb4d0bc4764d69c8', 'i-04c56ff92723f7e4d', 'i-0003276f9e230ae75', 'i-035c4ed1045261ca2', 'i-0e1adcbb78b500d65', 'i-07e1993c01537adc0', 'i-0611011ef0dfc6d11', 'i-0e8ef77ad5b1048c8', 'i-0881143b48abef8db', 'i-0ca350d10a23fea4b', 'i-0fb983668c619bf1a', 'i-0c507f34d62e057ea', 'i-09c8d94d5e2bc5524', 'i-038dff30149709526', 'i-0019860b456d81b35', 'i-033564c4ca8da657a', 'i-06f81f441cb990422', 'i-0f5a3a0816e8f3ef6', 'i-0523ee5152196f463', 'i-074d8d1e26556d861', 'i-05a854

INFO:__main__:Waiting for instances ['i-0e26a638cdf62e08b', 'i-0624b345eebba352b', 'i-039035e2661be0d81', 'i-0b1a81a710a9d8005', 'i-03a5bde8db6705ea8', 'i-032b589f66bed4eb0', 'i-023f9e6375eaeac3b', 'i-0330a450762ac712c', 'i-0c298d1cd02f46db3', 'i-0fe5b895eae68d432', 'i-09cfd1440d68a6d2b', 'i-0bc1f830df701fdd7', 'i-04fda51ec75253a15', 'i-00acdc98b08fa1e1a', 'i-031feaebda22819db', 'i-04555298f2982df1f', 'i-04c010a56d6d6e761', 'i-0a17b625c2b6ef575', 'i-001a265da60a3af9f', 'i-07df144d6d07b9e83', 'i-09c62b2d0387adff1', 'i-0fd8cddef6293da46', 'i-0853c238f478b7587', 'i-0625c6e2fd0466472', 'i-019910764e8f26c2f', 'i-0f8dd2416bbf48d5f', 'i-0ca532012af6362e1', 'i-0f5462e0892be2bbf', 'i-046430eb18de83e93', 'i-013eb5e6112f48492', 'i-000e547955300ab3c', 'i-0b2c7e406c83ffe6b', 'i-0f53cf19d2167e06d', 'i-03fa45a526a9adcf3', 'i-02040ab6405803fe2', 'i-06991e92b657e576d', 'i-0dc0bba7911be274d', 'i-0c625b529b716247a', 'i-0efb8af25cc91e885', 'i-0739a320c36fc84b5', 'i-044a2904684630cca', 'i-0abf6e383e02b9f77

INFO:__main__:Instances ['i-02831728b28de8402', 'i-00a56368b69bbcd77', 'i-0dc381be3d157b8c8', 'i-014027a5575546ae3', 'i-0a30aa0e756b9e43a', 'i-063af1578237f39ee', 'i-0e7e2640b61e5460d', 'i-049b1cbb15f7eb736', 'i-0af71663cb6c0908e', 'i-02e102bbd86585fdf', 'i-0c027a905d99a033b', 'i-00a39c6bc5cb997b8', 'i-08133732ee5167442', 'i-0b70c21cf6d5c3a8f', 'i-059b7b0ad6bd0aeb1', 'i-0fe49f29dd3e95316', 'i-0f42dae7121249d71', 'i-0fdf037a21799a7b3', 'i-08499d1e9cb16ed04', 'i-07a8201a5e34c674b', 'i-08cb270b6dc742ecf', 'i-0561ec045285a0034', 'i-076c7fd0e6f12da37', 'i-0a7866ecd5149062d', 'i-0eca5a317f8e3d209', 'i-0612ef84a7b5a6ade', 'i-050b46ccc37d1fe06', 'i-06fda5eb2bbdd634d', 'i-0ee7e23969ae03ac9', 'i-02a29744b755b40f0', 'i-04629813f094dfc22', 'i-0d752b1936eadfc1d', 'i-0cf352c84b95badc3', 'i-07613d1ce2df9087d', 'i-0d66c80c75e192556', 'i-0cd92603d04847ce8', 'i-058799ca1b646d115', 'i-06732073109455060', 'i-09667c6249cd20c87', 'i-0ef1366d7f45b8b27', 'i-0aecdccf748da4f7b', 'i-0acbc6f47509da32e', 'i-01dfd7

INFO:__main__:Waiting for instances ['i-03bde775b1f82a053', 'i-0b6e8414fff0eaea2', 'i-011642933d88ba106', 'i-09ecd4606fc4be89f', 'i-0df42bb3a782cf877', 'i-0a9697fa786f1c540', 'i-006fd9a79a803ee02', 'i-033162c9f80ee34a8', 'i-0d9029e42b0753c01', 'i-0c342fab474e4aec0', 'i-0c1323586c3293e0d', 'i-0caf40be31b7fdf1b', 'i-0412678617ff63930', 'i-089a71aa520368547', 'i-0d61f5b2a4f10606a', 'i-049b4753f8109f790', 'i-0733a1f7321f285e1', 'i-022d5d22bf2c181dc', 'i-05a188cc1f3eab4d7', 'i-0a131b0a4a3ad9970', 'i-0e9d17e455178abc2', 'i-0f223eeb0e3ad00b5', 'i-0af3372aabde6c4df', 'i-0f71041714f4cca13', 'i-0a2938d8f7a8efc6d', 'i-06bc332f3c7d7fa5c', 'i-0149f5bf9c99fe34d', 'i-084a19c5b88b14128', 'i-00ef77e8e2921a977', 'i-083a26fd3f2f9680b', 'i-0b58ecc54a079ae9f', 'i-097c14cc0648559f3', 'i-0f335f265f8c0b3d4', 'i-0bbd2b58923cc846f', 'i-06f3b6ccaa3ac668a', 'i-0b5afbcab29319fcd', 'i-07b37fa29e96dfb77', 'i-0c6141665b3c4c2ed', 'i-090d5c9637bccf39b', 'i-08e164347036da16a', 'i-064a9f862a68fb007', 'i-04b93bea2b229d16f

Because you are creating a very large volume of instances, it is possible that some failed. If failed, add more instances via the below script manually.

In [15]:
def create_instances1( total_nodes, port, REGIONS, node_count, shard_counts, fs,nodes_per_instance,rpt):
    instances_ = []
    instances_1 = []
    Assigned_ip=[]
    coordinator_address = ('10.0.1.79', 3000)  # Address of the coordinator node
    count_=6010
    print (f"There are {len(REGIONS)} regions.")
    total_nodes_=node_count
    for k, region in enumerate([REGIONS[16]]):
        for er1 in range (nodes_per_instance):
            ec2 = boto3.resource('ec2', region_name=region)
            ec2_c=boto3.client('ec2', region_name=region)
            print(region)
            new_ip=generate_random_ip(vpc_info[region]['cidr_block'])
            while (new_ip in Assigned_ip):
                new_ip=generate_random_ip(vpc_info[region]['cidr_block'])
            Assigned_ip.append(new_ip)
            temp_data=""
            for er in range (nodes_per_instance):
                temp_data+="nohup "+ start_node(count_+er, new_ip, port+er, coordinator_address)+" &\n"
            userdata='''#!/bin/bash
    sudo yum install -y python3
    sudo yum install -y python3-pip
   '''+ f"echo $'{escape_single_quotes(script_content_client)}' > /tmp/script.py \n nohup "+ temp_data
            try:
                instance = ec2.create_instances(
                    ImageId=ami_ids[region],  # Replace with a valid AMI ID
                    InstanceType='t3.micro',
                    MinCount=1,
                    MaxCount=1,
                    NetworkInterfaces=[{
                        'DeviceIndex': 0,
                        'AssociatePublicIpAddress': True,
                        'SubnetId': vpc_info[region]['subnet_id'],  # Replace with a valid Subnet ID
                        'Groups': [vpc_info[region]["sg_group"]],
                        'PrivateIpAddress':new_ip
                    }],
                    IamInstanceProfile={
                        'Name': 'Experiment_'  # Replace with the name of your IAM role
                    },
                UserData='''#!/bin/bash
                sudo yum install -y amazon-ssm-agent
sudo systemctl enable amazon-ssm-agent
sudo systemctl start amazon-ssm-agent''')[0]
            except:
                instance = ec2.create_instances(
                    ImageId=ami_ids[region],  # Replace with a valid AMI ID
                    InstanceType='t2.micro',
                    MinCount=1,
                    MaxCount=1,
                    NetworkInterfaces=[{
                        'DeviceIndex': 0,
                        'AssociatePublicIpAddress': True,
                        'SubnetId': vpc_info[region]['subnet_id'],  # Replace with a valid Subnet ID
                        'Groups': [vpc_info[region]["sg_group"]],
                        'PrivateIpAddress':new_ip
                    }],
                    IamInstanceProfile={
                        'Name': 'Experiment_'  # Replace with the name of your IAM role
                    },
                UserData='''#!/bin/bash
                sudo yum install -y amazon-ssm-agent
sudo systemctl enable amazon-ssm-agent
sudo systemctl start amazon-ssm-agent''')[0]
            count_+=nodes_per_instance
            total_nodes_-=nodes_per_instance
            instances_.append({'instance':instance,'region':region,'UserData': userdata})
            instances_1.append({'instance':instance,'region':region,'UserData': userdata})
    print (f"finished one region: {region}")
    threading.Thread(target=ex,args=(instances_1,region)).start()
    return instances_
print (regions[16])
create_instances1(5,35001,regions,5000,[1,17],[3,2],6,3)

sa-east-1
There are 17 regions.
sa-east-1
sa-east-1
sa-east-1
sa-east-1
sa-east-1
sa-east-1
finished one region: sa-east-1


[{'instance': ec2.Instance(id='i-02ba2430fc0f92cf0'),
  'region': 'sa-east-1',
  'UserData': '#!/bin/bash\n    sudo yum install -y python3\n    sudo yum install -y python3-pip\n   echo $\'import socket\nimport threading\nimport pickle\nimport random\nimport time\nimport hashlib\nimport math\nimport queue\nfrom concurrent.futures import ThreadPoolExecutor\nfrom collections import defaultdict\nfrom collections import deque\nimport base64\nimport argparse\nCHUNK_SIZE = 1024 \nRETRY_LIMIT = 15 \nRETRY_DELAY = 2   \nglobal_instance_list = []\n\ndef hash_content(content):\n    return hashlib.sha256(content.encode()).hexdigest()\nclass Node:\n    def __init__(self, node_id, address, coordinator_address):\n        self.node_id = node_id\n        self.address = address\n        self.shard_addresses = []  # List of addresses within the same shard\n        self.shard_id = None\n        self.is_primary = False\n        self.coordinator_address = coordinator_address\n        self.state = defaultdic

INFO:__main__:Instances ['i-0b82a49d40d53b474', 'i-0e6e2daa3687b0289', 'i-0b594c0a4444f0293', 'i-08fefbb2c1bd03265', 'i-0f4ef5bd5fce0e1dd', 'i-0c8a21c3228ba8017'] are ready
INFO:__main__:Waiting for instances ['i-02ba2430fc0f92cf0', 'i-07443c50ee74ae23a', 'i-0b8d6e298d7ed32d9', 'i-0979b260e919feeca', 'i-0a72dd9ef70328132', 'i-0325cfa6f6aba6e38'] to be in 'running' state and initialized


# After the experiment, terminate the instances.

In [15]:
# Set up logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

VALID_REGIONS = [
    'us-east-1', 'us-east-2', 'us-west-1', 'us-west-2',
    'af-south-1', 'ap-east-1', 'ap-south-1', 'ap-northeast-2',
    'ap-southeast-1', 'ap-southeast-2', 'ap-northeast-1',
    'ca-central-1', 'eu-central-1', 'eu-west-1', 'eu-west-2',
    'eu-south-1', 'eu-west-3', 'eu-north-1', 'me-south-1',
    'sa-east-1'
]

def delete_instances(instance_ids,default_region='eu-north-1'):
  # Specify a default region to create the initial client
    try:
        # Create a default EC2 client to get the instance details
        ec2_client = boto3.client('ec2', region_name=default_region)
        # Describe the instances to get their details
        response = ec2_client.describe_instances(InstanceIds=instance_ids)
        logger.info(f"Describe instances response: {response}")
        
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                instance_id = instance['InstanceId']
                availability_zone = instance['Placement']['AvailabilityZone']
                region = get_region_from_az(availability_zone)
                #logger.info(f"Instance {instance_id} is in availability zone {availability_zone}, region {region}")
                
                if region:
                    try:
                        # Create an EC2 client for the specific region
                        ec2 = boto3.client('ec2', region_name=region)
                        # Terminate the instance
                        ec2.terminate_instances(InstanceIds=[instance_id])
                        logger.info(f"Terminated instance {instance_id} in region {region}")
                    except Exception as e:
                        logger.error(f"Failed to terminate instance {instance_id}: {str(e)}")
                else:
                    logger.error(f"Could not determine a valid region from availability zone {availability_zone} for instance {instance_id}")
    except Exception as e:
        logger.error(f"Failed to describe instances: {str(e)}")

AttributeError: 'dict' object has no attribute 'id'

# Loop through the filtered instances to stop them.
After experiement, remove the instances.

In [4]:

for region in regions:
    ec2 = boto3.client('ec2', region_name=region)
    # Retrieve all instances
    response = ec2.describe_instances()
    instance_ids = []
    for reservation in response['Reservations']:
        for instance in reservation['Instances']:
            if instance['State']['Name'] != 'terminated' and instance['State']['Name'] != 'stopped':  # Only add non-terminated instances
                instance_ids.append(instance['InstanceId'])

    # Terminate instances
    if instance_ids:
        terminate_response = ec2.stop_instances(InstanceIds=instance_ids)
        waiter = ec2.get_waiter('instance_stopped')
        waiter.wait(InstanceIds=instance_ids)
        ec2.start_instances(InstanceIds=instance_ids)
    else:
        print("No active instances to terminate.")

ClientError: An error occurred (UnauthorizedOperation) when calling the StartInstances operation: You are not authorized to perform this operation. User: arn:aws:iam::533267193546:user/Experiment is not authorized to perform: ec2:StartInstances on resource: arn:aws:ec2:us-east-1:533267193546:instance/i-0048b03b294a768c2 because no identity-based policy allows the ec2:StartInstances action. Encoded authorization failure message: gM-fxgqMJcvbO2UMRv6nkai5TesSwyEf33iy1w7Xp1oT-UKZQ7bKWuVq7_sG2dzs0ZbpYaIoWDEUMq31tdP9ielR3BbgGowoy64Vtr32Km2GRnycWMZ6s7vYycXRRukCkQyXxMIq-JiJo2Koi2x8Rl8YymYnXFNoQg5lvS4TBxSWM_MTInSvPKvNpPjhGHRc7k9dYQRCZ7JhfFXBVDIEvXm7p8J6agwN8sb3U8sch6yu2T_vaTRczrOZio4N7W_dNSOLkmx-ZIsZ2xOm-IhcAldn-slfk5dhOnb1k8pCuhVEgsT1wBiMe7IYpEhP_mKJnsguHChUsHKBf5rvFRVnSTlhz-yz4ZyaaYKpDRfEbfksytzL_14BNAp4DnKgo5UvHxaDoci6LOQ6vT3eB4_uJK97-_ak6nq3qxaZ-MieKCh_3sESIi8LFVAohCPRAxxUujRLLrwHJoBx-Qmfvdip7XnhXd37YaJzpI8LNZO2pv0BWb3TBBrlLnpDitt7kHo9AXy1fOnfWXWrqY1AgHEUEt107RbRD4vMb1j14QdcHaKerOIJvtbwPBWA

In [19]:
for region in regions:
    ec2 = boto3.client('ec2', region_name=region)
    # Retrieve all instances
    response = ec2.describe_instances()
    instance_ids = []
    for reservation in response['Reservations']:
        for instance in reservation['Instances']:
            if instance['State']['Name'] != 'terminated':  # Only add non-terminated instances
                instance_ids.append(instance['InstanceId'])

    # Terminate instances
    if instance_ids:
        terminate_response = ec2.terminate_instances(InstanceIds=instance_ids)
        print("Termination requested for:", terminate_response['TerminatingInstances'])
    else:
        print("No active instances to terminate.")

No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
No active instances to terminate.
